In [1]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

m:\Third Year\Sixth Semester\Projects\NLP-Project\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import faiss 
import numpy as np
import pickle

In [3]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [4]:
class semanticEmbedding:
    def __init__(self, model_name='sentence-transformers/all-mpnet-base-v2'):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)

    def mean_pooling(self, model_output, attention_mask):
        token_embeddings = model_output[0] #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    def get_embedding(self, sentences):
    # Tokenize sentences
        encoded_input = self.tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
        with torch.no_grad():
            model_output = self.model(**encoded_input)
        # Perform pooling
        sentence_embeddings = self.mean_pooling(model_output, encoded_input['attention_mask'])

        # Normalize embeddings
        sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
        return sentence_embeddings.detach().numpy()

In [5]:
class FaissIdx:

    def __init__(self, model, dim=768):
        self.index = faiss.IndexFlatIP(dim) # need to load the pickle model in the final file
        # self.index = faiss
        # Maintaining the document data
        self.doc_map = dict()
        self.model = model
        self.ctr = 0

    def add_doc(self, document_text):
        self.index.add(self.model.get_embedding(document_text))
        self.doc_map[self.ctr] = document_text # store the original document text
        self.ctr += 1

    def search_doc(self, query, k=3):
        D, I = self.index.search(self.model.get_embedding(query), 5)
        return [{self.doc_map[idx]: score} for idx, score in zip(I[0], D[0]) if idx in self.doc_map]
    def save_index(self, index_filename, doc_map_filename):
        # Save Faiss index to file
        faiss.write_index(self.index, index_filename)

        # Save document map to file using pickle
        with open(doc_map_filename, 'wb') as f:
            pickle.dump(self.doc_map, f)

    def load_index(self, index_filename, doc_map_filename):
        # Load Faiss index from file
        self.index = faiss.read_index(index_filename)

        # Load document map from file using pickle
        with open(doc_map_filename, 'rb') as f:
            self.doc_map = pickle.load(f)

In [6]:
# creating an instance of the class
model = semanticEmbedding()

In [7]:
index = FaissIdx(model)

In [8]:
index.load_index('index.bin', 'doc_map.pkl')

In [9]:
import google.generativeai as genai

In [10]:
genai.configure(api_key=os.environ.get('GOOGLE_API_KEY'))

In [11]:
model = genai.GenerativeModel('gemini-pro')

# given the query, run the following code sequentially

In [22]:
query = "I want to help people in need"

In [23]:
user_name = 'Amar'

In [24]:
query_result = index.search_doc(query)

In [25]:
context = ''

In [26]:
for i in range(5):
    for key, value in query_result[i].items():
        if(value>0.23):
            context += key

In [27]:
context

'Grant-in-Aid Scheme for Assistance to Voluntary Organisations in the Field of Women and Child Development = The policy to give recognition promote, stimulate and develop Voluntary organisations and to provide opportunities to train their manpower so that the commitment of voluntary Organisation is mobilised for the well-being of the communit.Grant-in-aid scheme for voluntary agencies handling environment sectors = Get information about Grants-In-Aid scheme for voluntary agencies which have worked in the field of environment and related sectors provided by the National Afforestation and Eco-Development Board of Ministry of Environment and Forests. Information on guidelines, activities, quantum of grant, proportion of species for plantation, etc. can be accessed. Information can be accessed also on procedure for applications, scrutiny, sanction and evaluation of projects.General Grant-in-Aid Scheme in the field of Women and Child Development = The role of voluntary organizations and the

In [28]:
content = ""

In [34]:
if context == '':
    print('Is there anything else I can help you with?')
else:
    formatted_response = f'''Hello {user_name},

Thank you for reaching out! 🌟 Based on the information you provided, here are some government schemes that might be relevant to your situation:

1. [Scheme Name 1]:
   - Information like eligibility, benefit, how to apply link.

2. [Scheme Name 2]:
   - Information like eligibility, benefit, how to apply link.

3. [Scheme Name 3]:
   - Information like eligibility, benefit, how to apply link.

4. [Scheme Name 4]:
   - Information like eligibility, benefit, how to apply link.

5. [Scheme Name 5]:
   - Information like eligibility, benefit, how to apply link.

If you have any more questions or need further assistance, feel free to ask!

Best regards,
GovSevak.

*This message is generated by GENAI, GENAI can make mistakes. Consider checking important information.*

QUERY:${query}$ CONTEXT:${context}$'''

    response = model.generate_content(formatted_response)
    print(response.text)


Hello Amar,

Thank you for reaching out! 🌟 Based on the information you provided, here are some government schemes that might be relevant to your situation:

1. **Grant-in-Aid Scheme for Assistance to Voluntary Organisations in the Field of Women and Child Development**:
   - **Eligibility**: Voluntary organisations working in the field of women and child development.
   - **Benefit**: Financial assistance for various activities such as organising training programmes, awareness campaigns, and providing services to women and children.
   - **How to apply**: Contact the Ministry of Women and Child Development for more information and application procedures.

2. **Grant-in-Aid Scheme for Voluntary Agencies Handling Environment Sectors**:
   - **Eligibility**: Voluntary agencies working in the field of environment and related sectors.
   - **Benefit**: Financial assistance for activities such as afforestation, waste management, and environmental education.
   - **How to apply**: Contact th